In [1]:
import os
import pandas as pd
import glob

In [2]:
startdate, enddate = 20220101, 20230131 

In [3]:
directory_mask = os.path.join("../../data/9. data_for_forecasting/weather", "*209.txt")
directory_mask_wind_speed = os.path.join("../../data/9. data_for_forecasting/weather", "*060.txt")
# Find list of matching filenames in the directory
filenames = glob.glob(directory_mask)
filename_wind_speed = glob.glob(directory_mask_wind_speed)
# Add the wind speed data manually
filenames.append(filename_wind_speed[0])
dfs = []

if (len(filenames) > 0):
    df_list = []
    for filename in filenames:
        idf = pd.read_csv(filename, skiprows=19)
        idf.drop(columns=' SOUID', inplace=True)
        idf.rename(columns=lambda x: x.strip(), inplace = True)
        idf = idf[(idf['DATE'] >= startdate) & (idf['DATE'] <= enddate)]
        dfs.append(idf)


In [4]:
master_df = dfs[0]
for idf in dfs[1:]:
    master_df = master_df.merge(idf, how='outer', on='DATE')

In [5]:
#Change to datetime
master_df['DATE'] = master_df['DATE'].apply(lambda x: str(x)[:4]+'-'+str(x)[4:6]+'-'+str(x)[-2:])
master_df['DATE'] = pd.to_datetime(master_df['DATE'], format='%Y-%m-%d')

In [6]:
#Add powiat_voivod column and rename the columns
master_df['powiat_voivod'] = 'powiat warszawa, mazowieckie'
master_df.rename(columns={"CC": "cloud_cover", "Q_CC": "Q_cloud_cover", "FG": "wind_speed", "Q_FG":"Q_wind_speed",
                          "HU":"humidity", "Q_HU":"Q_humidity", "RR":"precipitation", "Q_RR":"Q_precipitation",
                          "SD":"snow_depth", "Q_SD":"Q_snow_depth", "TG":"temperature", "Q_TG":"Q_temperature"}, inplace=True)

#Recalculate the temperature
master_df['temperature'] /= 10

In [7]:
master_df.to_csv("../../data/9. data_for_forecasting/Combined_weather_data_forecasting.csv")